In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg
from paddleocr import PaddleOCR, draw_ocr

In [ ]:
FONT = "C:\\Users\\ADMIN\\Desktop\\Slide_School\\SlideKy7\\PBL6\\Preparation\\latin.ttf"

In [ ]:
detector = PaddleOCR(use_angle_cls = False, lang = "vi", use_gpu = False)

In [ ]:
config = Cfg.load_config_from_name('vgg_transformer')
config['cnn']['pretrained'] = True
config['predictor']['beamsearch'] = True
config['device'] = 'cpu' # mps

recognitor = Predictor(config)

In [ ]:
def predict(recognitor, detector, img_path, save_path, padding=4, dpi=100):
    img = cv2.imread(img_path)
    result = detector.ocr(img_path, cls=False, det=True, rec=False)
    result = result[:][:][0]

    boxes = []
    for line in result:
        boxes.append([[int(line[0][0]), int(line[0][1])], [int(line[2][0]), int(line[2][1])]])
    boxes = boxes[::-1]

    padding = 4
    for box in boxes:
        box[0][0] = box[0][0] - padding
        box[0][1] = box[0][1] - padding
        box[1][0] = box[1][0] + padding
        box[1][1] = box[1][1] + padding

    texts = []
    for box in boxes:
        cropped_image = img[box[0][1]:box[1][1], box[0][0]:box[1][0]]
        try:
            cropped_image = Image.fromarray(cropped_image)
        except:
            continue
        rec_result = recognitor.predict(cropped_image)
        text = rec_result
        texts.append(text)

    def get_rectangle_points(x1, y1, x2, y2):
        x_tl = x1
        y_tl = y2
        x_br = x2
        y_br = y1
        return [(x1, y1), (x2, y1), (x2, y2), (x1, y2)]
    _boxes = [get_rectangle_points(boxe[0][0], boxe[0][1], boxe[1][0], boxe[1][1]) for boxe in boxes]

    #img = draw_ocr(img, _boxes)
    img = draw_ocr(img, _boxes, texts, scores=None, font_path=FONT)
    img_name = img_path.split('/')[-1]
    cv2.imwrite(os.path.join(save_path, img_name), img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    height, width, depth = img.shape
    figsize = width / float(dpi), height / float(dpi)
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])
    ax.axis('off')
    ax.imshow(img, cmap='gray')

    plt.show()
    return boxes, texts

In [ ]:
input_path = './images/image4.jpg'
output_path = './output'
# Predict
boxes, texts = predict(recognitor, detector, input_path, output_path, padding=2, dpi=100)

In [ ]:
texts

In [2]:
class_colors = {
    "current_places": (255,0,0),
    "dob": (0,255,0),
    "expire_date": (0,0,255),
    "gender": (255,255,0),
    "id": (255,0,255),
    "name":(0,255,255),
    "nationality": (255, 128, 0),
    "origin_place": (0,0,0)
}

In [ ]:
def predict(recognitor, detector, input_path, padding = 2, dpi=100):
    img = cv2.cvtColor(cv2.imread(input_path), cv2.COLOR_BGR2RGB)
    detect = detector(img)
    class_indexes = detect[0].boxes.cls.numpy()
    class_names = [detector.names[int(class_index)] for class_index in class_indexes]
    boxes = detect[0].boxes.xyxy.numpy()
    
    result_dict = {name: [] for name in detector.names.value()}
    for i, box in enumerate(boxes):
        box[0] = box[0] - padding
        box[1] = box[1] - padding
        box[2] = box[2] + padding
        box[3] = box[3] + padding   
        x1, y1, x2, y2 = box
        crop_img = img[int(y1):int(y2), int(x1):int(x2)]
        crop_img_rgb = Image.fromarray(crop_img)
        color = class_colors.get(class_names[i], (255,255,255))
        text = recognitor.predict(crop_img_rgb)
        img = cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color = color, thickness=2)
        result_dict[class_names[i]].append(text)
        
    order_texts = [
        ", ".join(result_dict[detector.names[i]]) for i in sorted(detector.names)
    ]
    
    plt.figure(figsize=(6,6), dpi=dpi)
    plt.imshow(img)
    print(f"result are: {order_texts}")
    return boxes, order_texts